In [1]:
import pandas as pd
import matplotlib as rc
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.font_manager as fm
from prophet import Prophet
import numpy as np



In [4]:
file_path="./ref/lotto.xlsx"
ori = pd.read_excel(file_path, engine="openpyxl", header=None)  # 또는 engine="xlrd" (xls인 경우)
ori.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,회차별 추첨결과,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,년도,회차,추첨일,1등,NaN,2등,NaN,3등,NaN,4등,NaN,5등,NaN,당첨번호,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,1,2.0,3.0,4.0,5.0,6.0,보너스
3,2025,1173,2025.05.24,24,"1,179,946,063원",94,"50,210,471원",3139,"1,503,595원",163678,"50,000원",2689832,"5,000원",1,5.0,18.0,20.0,30.0,35.0,3
4,NaN,1172,2025.05.17,13,"2,203,004,452원",98,"48,705,881원",3059,"1,560,372원",152897,"50,000원",2590472,"5,000원",7,9.0,24.0,40.0,42.0,44.0,45


In [5]:

# 필요한 열만 선택해서 직접 이름 지정
df = ori.iloc[3:, 13:20]  # 예: 4번째 줄부터, 3~8번째 열만 사용
df.columns = ['번호1', '번호2', '번호3', '번호4', '번호5', '번호6','보너스']
df = df.dropna().astype(int)  # 또는 applymap(...) 사용 가능
df

,번호1,번호2,번호3,번호4,번호5,번호6,보너스
3,1,5,18,20,30,35,3
4,7,9,24,40,42,44,45
5,3,6,7,11,12,17,19
6,3,13,28,34,38,42,25
7,5,12,24,26,39,42,20
...,...,...,...,...,...,...,...
1171,16,24,29,40,41,42,3
1172,14,27,30,31,40,42,2
1173,11,16,19,21,27,31,30
1174,9,13,21,25,32,42,2


In [6]:
# 모든 번호를 하나의 배열로 평탄화 (flatten)
all_numbers = df.values.flatten()

# 등장 횟수 세기
number_counts = pd.Series(all_numbers).value_counts().sort_values(ascending=False)

# 가장 많이 나온 번호 8개
top8 = number_counts.head(8)

print(top8)


34    203
12    199
27    198
33    197
13    197
17    194
43    194
20    192
Name: count, dtype: int64


In [10]:
# 1. Series를 HTML 문자열로 변환 (인코딩은 하지 않음)
html_str = top8.to_frame(name="count").to_html()

# 2. 파일로 저장하면서 인코딩 설정



In [11]:
with open("lotto_top8.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [8]:
import random
import pandas as pd

# Lotto 번호가 있는 DataFrame의 이름이 df일 경우
all_numbers = df.values.flatten()
number_counts = pd.Series(all_numbers).value_counts().sort_values(ascending=False)

# 가장 많이 나온 번호 40개 기준 (top8이면 너무 좁음 → 분산 위해 40~45 추천)
top_numbers = number_counts.head(45).index.tolist()

# 추천 번호 담을 리스트
recommendations = []

for _ in range(10):
    # top_numbers 중에서 고르게 섞어서 6개 추천
    lotto = random.sample(top_numbers, 6)
    lotto.sort()  # 보기 좋게 정렬
    recommendations.append(lotto)

# 출력
for i, nums in enumerate(recommendations, 1):
    print(f"{i}번 추천 번호: {nums}")


low = [n for n in top_numbers if n <= 15]
mid = [n for n in top_numbers if 15 < n <= 30]
high = [n for n in top_numbers if n > 30]

recommendations = []
for _ in range(10):
    nums = random.sample(low, 2) + random.sample(mid, 2) + random.sample(high, 2)
    nums.sort()
    recommendations.append(nums)

# 출력
for i, nums in enumerate(recommendations, 1):
    print(f"{i}번 추천 번호 (균형형): {nums}")


1번 추천 번호: [15, 16, 32, 33, 38, 39]
2번 추천 번호: [2, 4, 11, 28, 30, 38]
3번 추천 번호: [4, 12, 13, 14, 21, 26]
4번 추천 번호: [2, 9, 12, 23, 29, 31]
5번 추천 번호: [7, 26, 28, 31, 33, 42]
6번 추천 번호: [7, 9, 10, 30, 37, 42]
7번 추천 번호: [4, 7, 18, 22, 25, 33]
8번 추천 번호: [7, 23, 28, 30, 44, 45]
9번 추천 번호: [1, 11, 23, 26, 38, 43]
10번 추천 번호: [10, 28, 30, 31, 33, 45]
1번 추천 번호 (균형형): [2, 9, 16, 23, 41, 42]
2번 추천 번호 (균형형): [11, 13, 18, 24, 42, 44]
3번 추천 번호 (균형형): [5, 7, 22, 25, 41, 43]
4번 추천 번호 (균형형): [4, 10, 19, 20, 40, 43]
5번 추천 번호 (균형형): [1, 14, 27, 29, 36, 42]
6번 추천 번호 (균형형): [1, 4, 19, 27, 43, 44]
7번 추천 번호 (균형형): [10, 13, 18, 24, 31, 32]
8번 추천 번호 (균형형): [1, 9, 24, 30, 40, 43]
9번 추천 번호 (균형형): [4, 12, 22, 24, 31, 40]
10번 추천 번호 (균형형): [9, 12, 16, 30, 33, 45]
